# A Cantera Simulation Using RMG-Py

In [ ]:
from rmgpy.chemkin import *
from rmgpy.tools.canteraModel import *
from rmgpy.species import Species
from IPython.display import display, Image

import os

In [ ]:
import matplotlib.pyplot as plt
plt.close('all')

Load the species and reaction from the RMG-generated chemkin file `chem_annotated.inp` and `species_dictionary.txt` file found in your `chemkin` folder after running a job.

In [ ]:
folder = '/home/mjliu/Documents/Models/Naphthalene/acetylene/run3'
speciesList, reactionList = loadChemkinFile(os.path.join(folder, 'chem0252.inp'),
                                            os.path.join(folder, 'species_dictionary.txt'))

Set a few conditions for how to react the system

In [ ]:
labels = ['A2', 'C2H2']
speciesDict = {}
for species in speciesList:
    if species.label in labels:
        speciesDict[species.label] = species
print speciesDict

In [ ]:
reactorTypeList = ['IdealGasConstPressureReactor']
molFracList=[{
        speciesDict['A2']: .5,
        speciesDict['C2H2']: .5,
    }]
Tlist = ([1200],'K')
Plist = ([16],'atm')
reactionTimeList = ([1], 's')

In [ ]:
# Create cantera object, loading in the species and reactions
job = Cantera(speciesList=speciesList, reactionList=reactionList, outputDirectory='temp')
# The cantera file must be created from an associated chemkin file

# We can either load the Model from the initialized set of rmg species and reactions
job.loadModel()

# Or load it from a chemkin file by uncommenting the following line:
#job.loadChemkinModel('data/minimal_model/chem_annotated.inp',transportFile='data/minimal_model/tran.dat')

# Generate the conditions based on the settings we declared earlier
job.generateConditions(reactorTypeList, reactionTimeList, molFracList, Tlist, Plist)
# Simulate and plot
alldata = job.simulate()
job.plot(alldata)

In [ ]:
# Show the plots in the ipython notebook
for i, condition in enumerate(job.conditions):
    print 'Condition {0}'.format(i+1)
    display(Image(filename="temp/{0}_mole_fractions.png".format(i+1)))

In [ ]:
# We can view the cantera model Solution's species and reactions
ctSpecies = job.model.species()
ctReactions = job.model.reactions()

# We can view a cantera species or reaction object from this
ct_ethane = ctSpecies[4]
ct_rxn = ctReactions[0]

print ct_ethane
print ct_rxn

In [ ]:
# We can also do things like modifying the cantera species thermo and reaction kinetics through modifying the
# RMG objects first, then using the `modifyReactionKinetics` or `modifySpeciesThermo` functions

# Alter the RMG objects in place, lets pick ethane and the first reaction
rmg_ethane = speciesDict[user_ethane]
rmg_ethane.thermo.changeBaseEnthalpy(2*4184)  # Change base enthalpy by 2 kcal/mol

rmg_rxn = reactionList[0]
rmg_rxn.kinetics.changeRate(4)  # Change A factor by multiplying by a factor of 4

# Take a look at the state of the cantera model before and after
print 'Cantera Model: Before'
ctSpecies = job.model.species()
ctReactions = job.model.reactions()
print 'Ethane Thermo = {} kcal/mol'.format(ctSpecies[4].thermo.h(300)/1000/4184)
print 'Reaction 1 Kinetics = {}'.format(ctReactions[0].rate)

# Now use the altered RMG objects to modify the kinetics and thermo
job.modifyReactionKinetics(0, rmg_rxn)
job.modifySpeciesThermo(4, rmg_ethane)

# If we modify thermo, the cantera model must be refreshed.  If only kinetics are modified, this does not need to be done.
job.refreshModel()

print ''
print 'Cantera Model: After'
ctSpecies = job.model.species()
ctReactions = job.model.reactions()
print 'Ethane Thermo = {} kcal/mol'.format(ctSpecies[4].thermo.h(300)/1000/4184)
print 'Reaction 1 Kinetics = {}'.format(ctReactions[0].rate)